# MultiplEYE preprocessing

In [1]:
from pathlib import Path
import polars as pl
import pandas as pd
import re
import json
import PIL
import math
from sanity import extract_informatio_file_path
from report import load_data, check_gaze, check_metadata, check_events, report_to_file as report_meta, plot_main_sequence
from functools import partial
from plot import plot_gaze


import pymovements as pm

In [43]:
# Create a dictionary with all the relevant information from the file path


path_asc_file = "data\\017_NL_NL_1_ET1_testrun_1733144369\\017nlnl1.asc"
#path_asc_file ="data\\016_NL_NL_1_ET1\\016nlnl1.asc"
#path_asc_file= "data\\010_ZH_CH_1_ET1\\010zhch1.asc"
#path_asc_file = "data\\003_HR_HR_1_ET1\\003hrhr1.asc"
local_file_part = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024"
#stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_ZH_CH_Zurich_1_2025"


logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\017_NL_NL_1_ET1_testrun_1733144369\logfiles\EXPERIMENT_LOGFILE_1_017_2024-12-02_1733144369.txt"
#logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\010_ZH_CH_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_010_2024-11-15_1731682963.txt"


    
vars_dict = extract_informatio_file_path(path_asc_file, local_file_part, stimulus_file_path, logfile_path)



Match was found at 5-52: 017_NL_NL_1_ET1_testrun_1733144369\017nlnl1.asc
Group 1 found at 5-8: 017
Group 2 found at 9-11: NL
Group 3 found at 12-14: NL
Group 4 found at 15-16: 1
Group 5 found at 17-20: ET1
Group 6 found at 20-39: _testrun_1733144369
Group 7 found at 40-48: 017nlnl1
C:\Users\saphi\PycharmProjects\multipleye-preprocessing\quality-report\output\NL_NL_Nijmegen_1_2024
True


In [44]:
vars_dict["asc_file"]


WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/017_NL_NL_1_ET1_testrun_1733144369/017nlnl1.asc')

In [47]:
vars_dict["completed_stimuli"] = Path(logfile_path).parent / "completed_stimuli.csv"
vars_dict["completed_stimuli"] 

WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/017_NL_NL_1_ET1_testrun_1733144369/logfiles/completed_stimuli.csv')

In [4]:
def report_to_file(message: str, report_file =vars_dict["report_file"]):
    with open(report_file, "a", encoding="utf-8") as report_file:
        report_file.write(f"{message}\n")
 

In [5]:
from stimulus import Stimulus
stimulus_dir = vars_dict["stimulus_file_path"]
lang = vars_dict["lang"].lower()

country = vars_dict["country"].lower()

labnum = vars_dict["labnum"]
stimulus_name = "PopSci_MultiplEYE"

stimulus = Stimulus.load(stimulus_dir, lang, country, labnum, stimulus_name)
for page in stimulus.pages:
    print(page.number, page.image_path)


1 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_1_nl.png
2 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_2_nl.png
3 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_3_nl.png
4 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_4_nl.png
5 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_5_nl.png
6 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_6_nl.png
7 C:\Users\saphi\Pycha

In [6]:

with open (vars_dict["json"]) as f:
    lab_config = json.load(f)
lab_config

{'Contact information': {'First_name': 'Stefan',
  'Last_name': 'Frank',
  'Affiliation': 'Radboud University',
  'Email': 'stefan.frank@ru.nl',
  'Phone': ''},
 'Data_collection_name': 'MultiplEYE_NL_NL_Nijmegen_1_2024',
 'Preregistration_core_or_additional_data_collection': 'pre-registration for core dataset',
 'Name_eye-tracker': 'EyeLink 1000 Plus',
 'Name_eye-tracker_manufacturer': 'SR Research',
 'Specification_camera_system': 'desktop mount',
 'Name_computer_operation_system': 'Windows 10',
 'Name_monitor': ['XL2430T', 'BenQ'],
 'Monitor_resolution_in_px': '(3840,2160)',
 'Screen_size_in_cm': '(59.8,33.6)',
 'Distance_in_cm': '60',
 'Script_direction': 'LTR',
 'Language_code': 'NL',
 'Country_code': 'NL',
 'Lab_number': '1',
 'City': 'Nijmegen',
 'Estimated_end_year_of_data_collection': '2024',
 'Use_of_multiple_devices': 'No',
 'Stimuli_preparation': ['Stimuli will be or has been prepared by our own lab'],
 'Psychometric_tests': {'Are_tests_conducted': 'No',
  'LWMC': '',
  'PL

## ASC to sample-level CSV

Convert the ASC files to CSV files (one for each page) where each row is a sample.

In [7]:
import importlib
stimulus_config_spec = importlib.util.spec_from_file_location(
        "stimulus_config", vars_dict["experiment_config"]
    )

stimulus_config = importlib.util.module_from_spec(stimulus_config_spec)
stimulus_config_spec.loader.exec_module(stimulus_config)

stimulus_config.IMAGE_WIDTH_PX

2375

In [7]:
from plot import load_data
from stimulus import load_stimuli
stimuli, lab_config = load_stimuli(vars_dict["stimulus_file_path"], vars_dict["lang"], vars_dict["country"], vars_dict["labnum"])
gaze = load_data(vars_dict['asc_file'], lab_config)

In [8]:
gaze.frame.filter(pl.col("activity") == "reading").unique("trial")


time,pupil,screen,trial,practice,stimulus,activity,pixel
i64,f64,str,str,bool,str,str,list[f64]
6080823,1116.0,"""page_1""","""trial_9""",false,"""Arg_PISACowsMilk_10""","""reading""","[254.1, 217.7]"
1458013,1316.0,"""page_1""","""PRACTICE_trial_2""",true,"""Enc_WikiMoon_13""","""reading""","[133.3, 218.7]"
1690040,1146.0,"""page_1""","""trial_1""",false,"""PopSci_MultiplEYE_1""","""reading""","[600.6, 338.7]"
3880477,1191.0,"""page_1""","""trial_5""",false,"""Lit_MagicMountain_6""","""reading""","[137.0, 197.1]"
3497287,1169.0,"""page_1""","""trial_4""",false,"""Lit_Alchemist_4""","""reading""","[89.0, 218.9]"
…,…,…,…,…,…,…,…
1335359,1178.0,"""page_1""","""PRACTICE_trial_1""",true,"""Lit_NorthWind_7""","""reading""","[1194.9, 877.0]"
5266595,1175.0,"""page_1""","""trial_7""",false,"""Lit_BrokenApril_9""","""reading""","[111.9, 203.6]"
6586049,1143.0,"""page_1""","""trial_10""",false,"""PopSci_Caveman_12""","""reading""","[108.8, 209.5]"


In [11]:
gaze.frame.filter(pl.col("stimulus") == "PopSci_Caveman_12").filter(pl.col("activity") == "rating").unique(pl.col("screen"))

time,pupil,screen,trial,practice,stimulus,activity,pixel
i64,f64,str,str,bool,str,str,list[f64]
6768213,1108.0,"""familiarity_rating_screen_1""","""trial_10""",false,"""PopSci_Caveman_12""","""rating""","[2254.3, 1613.5]"
6774223,1424.0,"""subject_difficulty_screen""","""trial_10""",false,"""PopSci_Caveman_12""","""rating""","[2242.8, 1613.1]"
6771429,1423.0,"""familiarity_rating_screen_2""","""trial_10""",false,"""PopSci_Caveman_12""","""rating""","[2289.8, 1689.9]"


In [12]:
gaze.frame

time,pupil,screen,trial,practice,stimulus,activity,pixel
i64,f64,str,str,bool,str,str,list[f64]
1335359,1178.0,"""page_1""","""PRACTICE_trial_1""",true,"""Lit_NorthWind_7""","""reading""","[1194.9, 877.0]"
1335361,1178.0,"""page_1""","""PRACTICE_trial_1""",true,"""Lit_NorthWind_7""","""reading""","[1194.3, 877.7]"
1335363,1179.0,"""page_1""","""PRACTICE_trial_1""",true,"""Lit_NorthWind_7""","""reading""","[1193.9, 877.7]"
1335365,1179.0,"""page_1""","""PRACTICE_trial_1""",true,"""Lit_NorthWind_7""","""reading""","[1194.3, 876.9]"
1335367,1180.0,"""page_1""","""PRACTICE_trial_1""",true,"""Lit_NorthWind_7""","""reading""","[1194.7, 875.7]"
…,…,…,…,…,…,…,…
6853775,1153.0,"""question_12132""","""trial_10""",false,"""PopSci_Caveman_12""","""question""","[2254.5, 1589.8]"
6853777,1153.0,"""question_12132""","""trial_10""",false,"""PopSci_Caveman_12""","""question""","[2254.5, 1588.2]"
6853779,1153.0,"""question_12132""","""trial_10""",false,"""PopSci_Caveman_12""","""question""","[2254.7, 1586.8]"


In [13]:
gaze._metadata


{'weekday': 'Mon',
 'month': 'Dec',
 'day': 2,
 'time': '00:15:26',
 'year': 2024,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v4.594 Jul  6 2012',
 'resolution': (2376, 1801),
 'tracking_mode': 'CR',
 'sampling_rate': 500.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'L',
 'mount_configuration': {'mount_type': 'Desktop',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'monocular',
  'short_name': 'MTABLER'},
 'pupil_data_type': 'AREA',
 'version_number': '4.594',
 'model': 'EyeLink 1000',
 'datetime': datetime.datetime(2024, 12, 2, 0, 15, 26),
 'calibrations': [{'timestamp': '1307711.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '1654534.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '2404756.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '2459282.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': '

In [16]:
lab_config

LabConfig(screen_resolution=(3840, 2160), screen_size_cm=(59.8, 33.6), screen_distance_cm=60.0)

In [15]:
report_file_2 = open(f"{vars_dict['output_dir']}/{vars_dict['participant_abbr']}_report2.txt", "w", encoding="utf-8")
report = partial(report_meta, report_file=report_file_2)
check_gaze(gaze, report)    
check_metadata(gaze._metadata, report)

In [28]:
from plot import preprocess
preprocess(gaze)


In [29]:
Path(vars_dict["local_file_path"]/ "data")

WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data')

In [30]:
my_dataset = pm.Dataset(dataset_definition, dataset_path)
print(my_dataset)

NameError: name 'dataset_definition' is not defined

In [31]:
gaze.events

shape: (32_675, 10)
┌────────────┬───────────┬───────────┬──────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ trial      ┆ stimulus  ┆ screen    ┆ name     ┆ … ┆ duration ┆ location  ┆ amplitude ┆ peak_velo │
│ ---        ┆ ---       ┆ ---       ┆ ---      ┆   ┆ ---      ┆ ---       ┆ ---       ┆ city      │
│ str        ┆ str       ┆ str       ┆ str      ┆   ┆ i64      ┆ list[f64] ┆ f64       ┆ ---       │
│            ┆           ┆           ┆          ┆   ┆          ┆           ┆           ┆ f64       │
╞════════════╪═══════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ PRACTICE_t ┆ Lit_North ┆ page_1    ┆ fixation ┆ … ┆ 556      ┆ [1191.534 ┆ null      ┆ null      │
│ rial_1     ┆ Wind_7    ┆           ┆          ┆   ┆          ┆ 767, 882. ┆           ┆           │
│            ┆           ┆           ┆          ┆   ┆          ┆ 389964]   ┆           ┆           │
│ PRACTICE_t ┆ Lit_North ┆ page_1    ┆ fixation ┆ … ┆ 132      ┆ [435.2

In [32]:
check_events(gaze, report)

In [33]:
plot_dir = vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_plots"
plot_dir.mkdir(exist_ok=True)

plot_main_sequence(gaze.events, plot_dir)

In [34]:
print(plot_dir)
test = (gaze.frame.filter(pl.col("stimulus") == "PopSci_Caveman_12").filter(pl.col("trial") == "trial_10"))
       
print(test)

C:\Users\saphi\PycharmProjects\multipleye-preprocessing\quality-report\output\NL_NL_Nijmegen_1_2024\017nlnl1_plots
shape: (119_531, 10)
┌─────────┬────────┬─────────────┬──────────┬───┬──────────┬─────────────┬────────────┬────────────┐
│ time    ┆ pupil  ┆ screen      ┆ trial    ┆ … ┆ activity ┆ pixel       ┆ position   ┆ velocity   │
│ ---     ┆ ---    ┆ ---         ┆ ---      ┆   ┆ ---      ┆ ---         ┆ ---        ┆ ---        │
│ i64     ┆ f64    ┆ str         ┆ str      ┆   ┆ str      ┆ list[f64]   ┆ list[f64]  ┆ list[f64]  │
╞═════════╪════════╪═════════════╪══════════╪═══╪══════════╪═════════════╪════════════╪════════════╡
│ 6586049 ┆ 1143.0 ┆ page_1      ┆ trial_10 ┆ … ┆ reading  ┆ [108.8,     ┆ [-25.17186 ┆ [0.312999, │
│         ┆        ┆             ┆          ┆   ┆          ┆ 209.5]      ┆ 7, -12.710 ┆ -0.252792] │
│         ┆        ┆             ┆          ┆   ┆          ┆             ┆ 677]       ┆            │
│ 6586051 ┆ 1143.0 ┆ page_1      ┆ trial_10 ┆ … ┆ readin

In [35]:
print(gaze.events.frame)
#gaze.frame.write_json(vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_gaze.json")
gaze.events.frame.write_json(vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_events.json")

shape: (32_675, 10)
┌────────────┬───────────┬───────────┬──────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ trial      ┆ stimulus  ┆ screen    ┆ name     ┆ … ┆ duration ┆ location  ┆ amplitude ┆ peak_velo │
│ ---        ┆ ---       ┆ ---       ┆ ---      ┆   ┆ ---      ┆ ---       ┆ ---       ┆ city      │
│ str        ┆ str       ┆ str       ┆ str      ┆   ┆ i64      ┆ list[f64] ┆ f64       ┆ ---       │
│            ┆           ┆           ┆          ┆   ┆          ┆           ┆           ┆ f64       │
╞════════════╪═══════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ PRACTICE_t ┆ Lit_North ┆ page_1    ┆ fixation ┆ … ┆ 556      ┆ [1191.534 ┆ null      ┆ null      │
│ rial_1     ┆ Wind_7    ┆           ┆          ┆   ┆          ┆ 767, 882. ┆           ┆           │
│            ┆           ┆           ┆          ┆   ┆          ┆ 389964]   ┆           ┆           │
│ PRACTICE_t ┆ Lit_North ┆ page_1    ┆ fixation ┆ … ┆ 132      ┆ [435.2

In [36]:
from plot import plot_gaze
for stimulus in stimuli:
    print(f"Plotting {stimulus.name}...")
    plot_gaze(gaze, stimulus, plot_dir)


Plotting PopSci_MultiplEYE...
Plotting Ins_HumanRights...
Plotting Ins_LearningMobility...
Plotting Lit_Alchemist...
Plotting Lit_MagicMountain...
Plotting Lit_Solaris...
Plotting Lit_BrokenApril...
Plotting Arg_PISACowsMilk...
Plotting Arg_PISARapaNui...
Plotting PopSci_Caveman...
Plotting Enc_WikiMoon...
Plotting Lit_NorthWind...


In [18]:

logfile= pl.read_csv(logfile_path, separator="\t")

In [19]:
logfile

timestamp,trial_number,stimulus_number,page_number,screen_onset_timestamp,keypress_timestamp,key_pressed,question,answer_correct,message
f64,str,str,str,str,str,str,str,str,str
12620.983,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** DATE:2024-11-15"""
12622.3842,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** EXP_START_TIMESTAMP:173168…"
12623.8452,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** SESSION_ID:1"""
12625.0668,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** PARTICIPANT_ID:010"""
12626.3405,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** DATASET_TYPE:pilot_session…"
…,…,…,…,…,…,…,…,…,…
2.8602e6,"""10""","""8""","""8131""","""2859479.7545999754""","""2860222.4987000227""","""down""","""True""","""False""","""FINAL ANSWER: correct answer i…"
2.8613e6,"""10""","""8""","""8132""","""2861320.9392998833""","""2861318.6283998657""","""down""","""True""","""True""","""preliminary answer: target_key"""
2.8616e6,"""10""","""8""","""8132""","""2861320.9392998833""","""2861630.635000067""","""space""","""True""","""True""","""preliminary answer: final_conf…"


In [20]:
stimulus_frame = logfile.filter(
              (pl.col("stimulus_number") == f"{11}")
            )
stimulus_frame.write_csv("stimulus_frame.csv")
stimulus_frame

timestamp,trial_number,stimulus_number,page_number,screen_onset_timestamp,keypress_timestamp,key_pressed,question,answer_correct,message
f64,str,str,str,str,str,str,str,str,str
529129.6903,"""3""","""11""","""<NA>""","""528118.9609998837""","""529129.6812999062""","""space""","""False""","""<NA>""","""optional break duration: 1010.…"
894145.5815,"""3""","""11""","""1""","""890073.0687999167""","""894145.571900066""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
901865.3043,"""3""","""11""","""2""","""899259.0349998791""","""901865.2959000319""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
909569.2674,"""3""","""11""","""3""","""906260.027299868""","""909569.2570998799""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
922841.2659,"""3""","""11""","""4""","""915718.9320998732""","""922841.2546999753""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
…,…,…,…,…,…,…,…,…,…
1.0751e6,"""3""","""11""","""11232""","""1074731.3721000683""","""1075050.253199879""","""space""","""True""","""False""","""preliminary answer: final_conf…"
1.0751e6,"""3""","""11""","""11232""","""1074373.4812000766""","""1075050.253199879""","""right""","""True""","""False""","""FINAL ANSWER: correct answer i…"
1.0762e6,"""3""","""11""","""11131""","""1076186.9310999755""","""1076184.9149998743""","""right""","""True""","""False""","""preliminary answer: distractor…"


In [21]:


print(stimuli)
def check_all_screens_logfile(logfile: pl, stimuli: Stimulus):
    for stimulus in stimuli:
        print(f"Checking {stimulus.name} in Logfile")
        stimulus_frame = logfile.filter(
              (pl.col("trial_number") == f"{stimulus.id}")
            )
        # check if all pages are present        
        for page in stimulus.pages:
            if f"{page.number}" not in stimulus_frame["page_number"].to_list():
                print(f"Missing page {stimulus.name} {page.number} in Logfile")  
                report_to_file(f"Missing page {stimulus.name} {page.number} in Logfile")
        # check if all questions are present              
        for question in stimulus.questions:
            if f"{question.id}" not in stimulus_frame["page_number"].to_list() and f"{question.id[1:]}" not in stimulus_frame["page_number"].to_list():
                print(f"Missing question_{question.id}")
                report_to_file(f"Missing question_{question.id} in Logfile")
                print(stimulus_frame["screen"])
                
        for instruction in stimulus.instructions:
            if f"{instruction.name}" not in stimulus_frame["page_number"].to_list():
                print(f"Missing instruction {instruction.name}")
                report_to_file(f"Missing instruction {instruction.name} in Logfile")
               
check_all_screens_logfile(logfile, stimuli)

[Stimulus(id=1, name='PopSci_MultiplEYE', type='experiment', pages=[StimulusPage(number=1, text='Het MultiplEYE-project\r\n\r\nDe naam "MultiplEYE" is een woordspeling die "multilingualism" of "multiple languages" (Engels voor "meertaligheid") combineert met "eye" (Engels voor "oog") van "eye-tracking" (oogbewegingsmeting). MultiplEYE is een door de Europese Unie gefinancierde COST-actie. COST-acties zijn onderzoeksnetwerken die worden ondersteund door de European Cooperation in Science and Technology, kortweg COST. Als financieringsorganisatie ondersteunt COST ons groeiende netwerk van onderzoekers in heel Europa en daarbuiten, door financiële steun te verlenen voor het uitvoeren van verschillende netwerkactiviteiten.', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024/stimuli_images_NL_NL_1/popsci_multipleye_id1_page_1_NL.png')), StimulusPage(number=2, text='Onder deze activiteiten zijn vergaderingen van werk

ColumnNotFoundError: "screen" not found

In [22]:
## check if all pages screens ect are present


def check_all_screens(gaze, stimuli):
    """ checking if all screens, where ET data is tracked are present in the gaze data frame, which is based on the ASC file
    it checks for all stimuli, if all pages and questions screens are present;it does not check for valditaion, calibration, instructions, etc.
    """
    for stimulus in stimuli:
        print(f"Checking {stimulus.name}")
        stimulus_frame = gaze.frame.filter(
              (pl.col("stimulus") == f"{stimulus.name}_{stimulus.id}")
            ).unique("screen")
        # check if all pages are present        
        for page in stimulus.pages:
            if f"page_{page.number}" not in stimulus_frame["screen"].to_list():
                print(f"Missing page {page.number}")  
                report_to_file(f"Missing page {page.number} in asc file")
        # check if all questions are present              
        for question in stimulus.questions:
            if f"question_{question.id}" not in stimulus_frame["screen"].to_list() and f"question_{question.id[1:]}" not in stimulus_frame["screen"].to_list():
                print(f"Missing question_{question.id}")
                report_to_file(f"Missing question_{question.id} in asc file")
                print(stimulus_frame["screen"])
                
        for rating in stimulus.ratings:
            if f"{rating.name}" not in stimulus_frame["screen"].to_list():
                print(f"Missing instruction {rating.name}")
                report_to_file(f"Missing rating {rating.name} in asc file")
                                
check_all_screens(gaze, stimuli)

Checking PopSci_MultiplEYE
Checking Ins_HumanRights
Checking Ins_LearningMobility
Checking Lit_Alchemist
Checking Lit_MagicMountain
Checking Lit_Solaris
Checking Lit_BrokenApril
Checking Arg_PISACowsMilk
Checking Arg_PISARapaNui
Checking PopSci_Caveman
Checking Enc_WikiMoon
Checking Lit_NorthWind


In [23]:
def load_messages(asc_file: Path, REGEX):
    with open(asc_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    messages = []
    for line in lines:
        match = re.match(REGEX, line)
        if match:
            messages.append(match.groupdict())
    return messages

REGEX = r'MSG\s+(?P<timestamp>\d+[.]?\d*)\s+(?P<message>\w+)'


messages = load_messages(vars_dict["asc_file"], REGEX)

In [24]:
print(messages)

[{'timestamp': '404044.0', 'message': 'var'}, {'timestamp': '404096.0', 'message': 'welcome_screen'}, {'timestamp': '1124828.0', 'message': 'informed_consent_screen'}, {'timestamp': '1129138.0', 'message': 'start_experiment'}, {'timestamp': '1129138.0', 'message': 'stimulus_order_version'}, {'timestamp': '1129140.0', 'message': 'showing_instruction_screen_1'}, {'timestamp': '1131992.0', 'message': 'showing_instruction_screen_2'}, {'timestamp': '1151361.0', 'message': 'showing_instruction_screen_3'}, {'timestamp': '1160379.0', 'message': 'camera_setup_screen'}, {'timestamp': '1172095.0', 'message': 'DISPLAY_COORDS'}, {'timestamp': '1270732.0', 'message': 'practice_text_starting_screen'}, {'timestamp': '1284445.0', 'message': 'TRIAL_VAR_LABELS'}, {'timestamp': '1284445.0', 'message': 'V_TRIAL_GROUPING'}, {'timestamp': '1284446.0', 'message': 'TRIALID'}, {'timestamp': '1285412.0', 'message': 'RECCFG'}, {'timestamp': '1285412.0', 'message': 'ELCLCFG'}, {'timestamp': '1285412.0', 'message':

In [25]:
stimulus = stimuli[2]
stimulus.name, stimulus.pages[0]

('Ins_LearningMobility',
 StimulusPage(number=1, text='VERSLAG VAN DE COMMISSIE AAN DE RAAD: Voortgangsverslag over een benchmark betreffende leermobiliteit\r\n\r\n1. INLEIDING\r\nIn de recente mededeling van de Commissie "Investeren in de jongeren van Europa" wordt het positieve effect van leermobiliteit op inzetbaarheid en actief burgerschap benadrukt. Leren, studeren en een opleiding volgen in een ander land is een unieke ervaring en kan nieuwe horizonten openen. ', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024/stimuli_images_NL_NL_1/ins_learningmobility_id3_page_1_NL.png')))

In [101]:
# check practice trials
def check_instructions(messages, stimuli):
    
    messages_only = [d.get('message') for d in messages]
    one_time_screens = ['welcome_screen', 'informed_consent_screen', 'start_experiment', 'stimulus_order_version', 'showing_instruction_screen_1', 'showing_instruction_screen_2', 'showing_instruction_screen_3', 'camera_setup_screen', 'practice_text_starting_screen', 'start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1','stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1', 'start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_2', 'stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_2','transition_screen', 'final_validation', 'show_final_screen', 'obligatory_break', 'obligatory_break_end']
    optional_screens = ['empty_screen', 'optional_break_screen']
    reoccuring_screens = ['showing_subject_difficulty_screen', 'showing_familiarity_rating_screen_1','showing_familiarity_rating_screen_2']
    reoccuring_msg = ['start_recording', 'stop_recording']
    
    def _check_validation_screen(last_index, index_next_stimulus):
        if "validation_before_stimulus" not in messages_only[last_index:index_next_stimulus]:
            print("Missing validation screen")
            report_to_file("Missing validation screen in asc file")
        else:
            print("Validation screen found")
  
    def _get_stimulus(id, next_id=None):
        right_stimulus = None
        next_stimulus = None
        for stimulus in stimuli:
            if stimulus.id == id:
                right_stimulus = stimulus
            if stimulus.id == next_id:
                next_stimulus = stimulus
       
        return right_stimulus, next_stimulus
            
    def _check_instruction_screens(last_index, index_next_stimulus):
        for instruction in reoccuring_screens:
            if f"{instruction}" not in messages_only[last_index:index_next_stimulus]:
                print(f"Missing instruction {instruction}")
                report_to_file(f"Missing instruction {instruction} in asc file")
            else:
                print(f"Rating {instruction} screens found")
                
    def _reoccuring_msg(trial):
        pattern = f"_trial_{trial}_stimulus_{stimulus.name}_{stimulus.id}"
        last_index = messages_only.index(f"start_recording{pattern}_page_1")
        if next_stimulus:
            index_next_stimulus = messages_only.index(f"start_recording_trial_{trial+1}_stimulus_{next_stimulus.name}_{next_stimulus.id}_page_1")
        else:
            index_next_stimulus = len(messages_only)
        
       
                
        msg_to_find = ["start_recording","screen_image_onset", "screen_image_offset", "stop_recording"]
        
        for page in stimulus.pages:
            for msg in msg_to_find:
                if msg.startswith("screen"):
                    current_pattern = f"page_{msg}"
                else:
                    current_pattern = f"{msg}{pattern}_page_{page.number}"
                    
                if current_pattern not in messages_only[last_index:index_next_stimulus]:
                    print(f"Missing {current_pattern} Messages in ASC file")
                else:
                    
                    last_index = messages_only[last_index:index_next_stimulus].index(current_pattern) + last_index
                            
                    print(f"{current_pattern} {last_index} found")
        _check_instruction_screens(last_index, index_next_stimulus)
        _check_validation_screen(last_index, index_next_stimulus)
                
    for trial, id in enumerate(stimulus_order[2:]): # for id in vars_dict["stimulus_order]: # skip th e practice trials
        try:
            stimulus, next_stimulus = _get_stimulus(id, stimulus_order[2:][trial+1]) # also get next stimulus to check for the reoccuring screens
        except IndexError:
            print("Last stimulus")
            stimulus, next_stimulus = _get_stimulus(id, None)
            
        print(f"Checking {stimulus.name}")
        _reoccuring_msg(trial+1) # add one because the first trial  has number 1 not 0 
       

               
       
        
                
    for one_time_screen in one_time_screens:
        if f"{one_time_screen}" not in messages_only:
            print(f"Missing screen {one_time_screen}")
            report_to_file(f"Missing one time screen {one_time_screen} in asc file")
        else:
            print(f"{one_time_screen} found")
            
    for optional_screen in optional_screens:
        if f"{optional_screen}" in messages_only:
            report_to_file(f"Optional screen {optional_screen} found in asc file")
            print(f"{optional_screen} found")

check_instructions(messages, stimuli)


Checking PopSci_MultiplEYE
start_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_1 196 found
page_screen_image_onset 203 found
page_screen_image_offset 204 found
stop_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_1 205 found
start_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_2 213 found
page_screen_image_onset 220 found
page_screen_image_offset 221 found
stop_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_2 222 found
start_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_3 230 found
page_screen_image_onset 237 found
page_screen_image_offset 238 found
stop_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_3 239 found
start_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_4 247 found
page_screen_image_onset 254 found
page_screen_image_offset 255 found
stop_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_4 256 found
start_recording_trial_1_stimulus_PopSci_MultiplEYE_1_page_5 264 found
page_screen_image_onset 271 found
page_screen_image_offset 272 found
st

In [56]:
stimulus_order= pl.read_csv(vars_dict["completed_stimuli"], separator=",")
print(stimulus_order)
stimulus_order = stimulus_order["stimulus_id"].to_list()
stimulus_order

shape: (12, 5)
┌───────────────────┬─────────────────────┬─────────────┬──────────────────────┬───────────┐
│ timestamp_started ┆ timestamp_completed ┆ stimulus_id ┆ stimulus_name        ┆ completed │
│ ---               ┆ ---                 ┆ ---         ┆ ---                  ┆ ---       │
│ f64               ┆ f64                 ┆ i64         ┆ str                  ┆ i64       │
╞═══════════════════╪═════════════════════╪═════════════╪══════════════════════╪═══════════╡
│ 908673.3011       ┆ 1.0757e6            ┆ 7           ┆ Lit_NorthWind        ┆ 1         │
│ 1.0757e6          ┆ 1.2282e6            ┆ 13          ┆ Enc_WikiMoon         ┆ 1         │
│ 1.3080e6          ┆ 1.9941e6            ┆ 1           ┆ PopSci_MultiplEYE    ┆ 1         │
│ 2.1116e6          ┆ 2.5541e6            ┆ 2           ┆ Ins_HumanRights      ┆ 1         │
│ 2.6013e6          ┆ 2.9842e6            ┆ 3           ┆ Ins_LearningMobility ┆ 1         │
│ …                 ┆ …                   ┆ …          

[7, 13, 1, 2, 3, 4, 6, 8, 9, 11, 10, 12]

In [40]:
def check_experiment_order(messages, stimulus_order):

    def _get_stimulus(id):
        for stimulus in stimuli:
            if stimulus.id == id:
                return stimulus
            
    for id in stimulus_order: # for id in vars_dict["stimulus_order]:
            stimulus = _get_stimulus(id)
            print(f"Checking {stimulus.name}")
            

check_experiment_order(messages, stimulus_order)         

Stimulus(id=2, name='Ins_HumanRights', type='experiment', pages=[StimulusPage(number=1, text='Universele Verklaring van de Rechten van de Mens - Preambule\r\n\r\nOverwegende, dat erkenning van de inherente waardigheid en van de gelijke en onvervreemdbare rechten van alle leden van de mensengemeenschap grondslag is voor de vrijheid, gerechtigheid en vrede in de wereld;', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024/stimuli_images_NL_NL_1/ins_humanrights_id2_page_1_NL.png')), StimulusPage(number=2, text='Overwegende, dat terzijdestelling van en minachting voor de rechten van de mens geleid hebben tot barbaarse handelingen, die het geweten van de mensheid geweld hebben aangedaan en dat de komst van een wereld, waarin de mensen vrijheid van meningsuiting en geloof zullen genieten, en vrij zullen zijn van vrees en gebrek, is verkondigd als het hoogste ideaal van iedere mens;', image_path=WindowsPath('C:/Users/s

In [38]:
messages_only = [d.get('message') for d in messages]
messages_only

['var',
 'welcome_screen',
 'informed_consent_screen',
 'start_experiment',
 'stimulus_order_version',
 'showing_instruction_screen_1',
 'showing_instruction_screen_2',
 'showing_instruction_screen_3',
 'camera_setup_screen',
 'DISPLAY_COORDS',
 'practice_text_starting_screen',
 'TRIAL_VAR_LABELS',
 'V_TRIAL_GROUPING',
 'TRIALID',
 'RECCFG',
 'ELCLCFG',
 'GAZE_COORDS',
 'THRESHOLDS',
 'ELCL_PROC',
 'ELCL_PCR_PARAM',
 'fixation_trigger',
 'fixation_trigger',
 'DISPLAY_COORDS',
 'VALIDATE',
 'VALIDATE',
 'VALIDATE',
 'VALIDATE',
 'VALIDATE',
 'VALIDATE',
 'VALIDATE',
 'VALIDATE',
 'VALIDATE',
 'start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1',
 'RECCFG',
 'ELCLCFG',
 'GAZE_COORDS',
 'THRESHOLDS',
 'ELCL_PROC',
 'ELCL_PCR_PARAM',
 'page_screen_image_onset',
 'page_screen_image_offset',
 'stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1',
 'RECCFG',
 'ELCLCFG',
 'GAZE_COORDS',
 'THRESHOLDS',
 'ELCL_PROC',
 'ELCL_PCR_PARAM',
 'fixation_trigger',
 'start_recor

In [ ]:
for instruction in stimuli[0][0].instructions:
    print(instruction.name)

In [ ]:
 for instruction in ["practice_trial_screen_1", "practice_trial_screen_2"]:
            if f"showing_{instruction}" not in messages_only:
                print(f"Missing instruction {instruction}")
                report_to_file(f"Missing instruction {instruction} in asc file")
            else:
                print(f"Rating {instruction} found")